<a href="https://colab.research.google.com/github/shishir-py/ml-projects/blob/main/catboost_with_GridCV_Feature_Imputation_with_a_Heat_Flux_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
df_path="/content/drive/MyDrive/Proj/kaggle competation/data.csv"
sub_path="/content/drive/MyDrive/Proj/kaggle competation/sample_submission.csv"

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [3]:
df=pd.read_csv(df_path)

In [4]:
df.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8


In [5]:
df=df.drop(['id','author'],axis=1)

In [6]:
df.columns

Index(['geometry', 'pressure [MPa]', 'mass_flux [kg/m2-s]', 'x_e_out [-]',
       'D_e [mm]', 'D_h [mm]', 'length [mm]', 'chf_exp [MW/m2]'],
      dtype='object')

In [11]:
df.isnull().sum()

geometry                   0
pressure [MPa]             0
mass_flux [kg/m2-s]        0
x_e_out [-]            10415
D_e [mm]                   0
D_h [mm]                   0
length [mm]                0
chf_exp [MW/m2]            0
dtype: int64

In [8]:
df['geometry']=df['geometry'].fillna('Unknown')

In [10]:
# This will fill the missing values in "pressure" and max flux  column with the mean of the respective "geometry" group
df['pressure [MPa]'] = df.groupby('geometry')['pressure [MPa]'].apply(lambda x: x.fillna(x.mean()))
df['mass_flux [kg/m2-s]'] = df.groupby('geometry')['mass_flux [kg/m2-s]'].apply(lambda x: x.fillna(x.mean()))
df['D_e [mm]'] = df.groupby('geometry')['D_e [mm]'].apply(lambda x: x.fillna(x.mean()))
df['D_h [mm]'] = df.groupby('geometry')['D_h [mm]'].apply(lambda x: x.fillna(x.mean()))
df['length [mm]'] = df.groupby('geometry')['length [mm]'].apply(lambda x: x.fillna(x.mean()))





In [15]:
categorical_cols = ['geometry']
preprocessor = ColumnTransformer([('encoder', OneHotEncoder(), categorical_cols)],
                                 remainder='passthrough')
data_encoded = preprocessor.fit_transform(df)

# Retrieve column names after one-hot encoding
encoded_categorical_cols = preprocessor.named_transformers_['encoder'].get_feature_names_out()
all_cols = list(encoded_categorical_cols) + list(df.columns.drop(categorical_cols))

# Convert the encoded data back to a DataFrame
data_normalized = pd.DataFrame(data_encoded, columns=all_cols)

# Normalize the numerical columns
numerical_cols = ['pressure [MPa]', 'mass_flux [kg/m2-s]', 'x_e_out [-]', 'D_e [mm]', 'D_h [mm]', 'length [mm]', 'chf_exp [MW/m2]']
data_normalized[numerical_cols] = (data_normalized[numerical_cols] - data_normalized[numerical_cols].mean()) / data_normalized[numerical_cols].std()

# Derive new features
data_normalized['D_e_ratio_D_h'] = data_normalized['D_e [mm]'] / data_normalized['D_h [mm]']
data_normalized['length_ratio_D_h'] = data_normalized['length [mm]'] / data_normalized['D_h [mm]']
data_normalized['pressure_mass_flux'] = data_normalized['pressure [MPa]'] * data_normalized['mass_flux [kg/m2-s]']
data_normalized['length_mass_flux'] = data_normalized['length [mm]'] * data_normalized['mass_flux [kg/m2-s]']



In [17]:
df.columns

Index(['geometry', 'pressure [MPa]', 'mass_flux [kg/m2-s]', 'x_e_out [-]',
       'D_e [mm]', 'D_h [mm]', 'length [mm]', 'chf_exp [MW/m2]'],
      dtype='object')

In [21]:
print(data_normalized.head())

   geometry_Unknown  geometry_annulus  geometry_plate  geometry_tube  \
0               0.0               0.0             0.0            1.0   
1               0.0               0.0             0.0            1.0   
2               1.0               0.0             0.0            0.0   
3               0.0               1.0             0.0            0.0   
4               0.0               0.0             0.0            1.0   

   pressure [MPa]  mass_flux [kg/m2-s]  x_e_out [-]  D_e [mm]  D_h [mm]  \
0       -0.896884             0.427551     1.742654 -0.046875 -0.176981   
1        0.129570             1.815175    -0.407757  0.353697 -0.202790   
2        0.777124            -0.629454     0.336464 -0.196238 -0.336996   
3        0.777124             0.372144    -0.271994 -0.640417  0.050136   
4        0.777124            -1.450216          NaN  0.522908 -0.161496   

   length [mm]  chf_exp [MW/m2]  D_e_ratio_D_h  length_ratio_D_h  \
0    -0.629188        -0.099287       0.264857  

In [22]:
data_normalized.isnull().sum()

geometry_Unknown           0
geometry_annulus           0
geometry_plate             0
geometry_tube              0
pressure [MPa]             0
mass_flux [kg/m2-s]        0
x_e_out [-]            10415
D_e [mm]                   0
D_h [mm]                   0
length [mm]                0
chf_exp [MW/m2]            0
D_e_ratio_D_h              0
length_ratio_D_h           0
pressure_mass_flux         0
length_mass_flux           0
dtype: int64

In [61]:
# Split the data into two sets based on the presence of missing values in the target column
data=data_normalized.copy()
test = data[data['x_e_out [-]'].isnull()]
train = data[~data['x_e_out [-]'].isnull()]


In [32]:
test.shape

(10415, 15)

In [33]:
train.head()

,geometry_Unknown,geometry_annulus,geometry_plate,geometry_tube,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],D_e_ratio_D_h,length_ratio_D_h,pressure_mass_flux,length_mass_flux
0,0.0,0.0,0.0,1.0,-0.896884,0.427551,1.742654,-0.046875,-0.176981,-0.629188,-0.099287,0.264857,3.555109,-0.383464,-0.269010
1,0.0,0.0,0.0,1.0,0.129570,1.815175,-0.407757,0.353697,-0.202790,-0.112573,1.211202,-1.744155,0.555122,0.235193,-0.204340
2,1.0,0.0,0.0,0.0,0.777124,-0.629454,0.336464,-0.196238,-0.336996,-0.590050,-0.653725,0.582316,1.750911,-0.489164,0.371409
3,0.0,1.0,0.0,0.0,0.777124,0.372144,-0.271994,-0.640417,0.050136,2.035290,-0.401708,-12.773475,40.595009,0.289202,0.757421
5,1.0,0.0,0.0,0.0,1.627687,0.353269,-0.700094,0.006938,-0.636379,-0.215896,-0.099287,-0.010902,0.339257,0.575011,-0.076269


In [27]:
pip install pandas scikit-learn catboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 7.9 MB/s eta 0:00:00


In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [34]:
# Split the data into features (X) and target variable (y)
X = train.drop('x_e_out [-]', axis=1)
y = train['x_e_out [-]']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the models
linear_reg = LinearRegression()
decision_tree = DecisionTreeRegressor()
random_forest = RandomForestRegressor()
catboost = CatBoostRegressor(verbose=False)
svm = SVR()

# Train the models
linear_reg.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
random_forest.fit(X_train, y_train)
catboost.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Make predictions on the test set
linear_reg_pred = linear_reg.predict(X_test)
decision_tree_pred = decision_tree.predict(X_test)
random_forest_pred = random_forest.predict(X_test)
catboost_pred = catboost.predict(X_test)
svm_pred = svm.predict(X_test)

# Evaluate the models
linear_reg_mse = mean_squared_error(y_test, linear_reg_pred)
linear_reg_r2 = r2_score(y_test, linear_reg_pred)

decision_tree_mse = mean_squared_error(y_test, decision_tree_pred)
decision_tree_r2 = r2_score(y_test, decision_tree_pred)

random_forest_mse = mean_squared_error(y_test, random_forest_pred)
random_forest_r2 = r2_score(y_test, random_forest_pred)

catboost_mse = mean_squared_error(y_test, catboost_pred)
catboost_r2 = r2_score(y_test, catboost_pred)

svm_mse = mean_squared_error(y_test, svm_pred)
svm_r2 = r2_score(y_test, svm_pred)

# Print the evaluation results
print("Linear Regression:")
print("MSE:", linear_reg_mse)
print("R^2:", linear_reg_r2)
print()

print("Decision Tree:")
print("MSE:", decision_tree_mse)
print("R^2:", decision_tree_r2)
print()

print("Random Forest:")
print("MSE:", random_forest_mse)
print("R^2:", random_forest_r2)
print()

print("CatBoost:")
print("MSE:", catboost_mse)
print("R^2:", catboost_r2)
print()

print("SVM:")
print("MSE:", svm_mse)
print("R^2:", svm_r2)
print()

Linear Regression:
MSE: 0.74359379789827
R^2: 0.25454504313130943

Decision Tree:
MSE: 1.0881180101185506
R^2: -0.09084148710441609

Random Forest:
MSE: 0.6316197017087882
R^2: 0.3667994020047549

CatBoost:
MSE: 0.5587627301826809
R^2: 0.4398387290137843

SVM:
MSE: 0.7280799224316653
R^2: 0.2700977486534797



In [49]:
# Define the parameter grid for grid search
param_grid = {
    'learning_rate': [0.01, 0.1],
    'depth': [6, 8, 10],
    'iterations': [100, 200, 300]
}

# Initialize the CatBoostRegressor
catboost = CatBoostRegressor(verbose=False)

# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(estimator=catboost, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_catboost = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the test set using the best model
catboost_pred = best_catboost.predict(X_test)

# Evaluate the best model
mse = mean_squared_error(y_test, catboost_pred)
r2 = r2_score(y_test, catboost_pred)

# Print the best hyperparameters and evaluation results
print("Best Hyperparameters:", best_params)
print("MSE:", mse)
print("R^2:", r2)

Best Hyperparameters: {'depth': 6, 'iterations': 300, 'learning_rate': 0.1}
MSE: 0.5562072457897207
R^2: 0.44240060959067595


In [65]:
# Initialize the CatBoostRegressor with the best hyperparameters
catboost = CatBoostRegressor(depth=6, iterations=300, learning_rate=0.1, verbose=False)

# Train the CatBoostRegressor model on the training set
catboost.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = catboost.predict(X_train)

# Make predictions on the test set
y_test_pred = catboost.predict(X_test)

# Evaluate the model on the training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Evaluate the model on the test set
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Print the evaluation results
print("Training set:")
print("MSE:", train_mse)
print("R^2:", train_r2)
print("")

print("Test set:")
print("MSE:", test_mse)
print("R^2:", test_r2)

Training set:
MSE: 0.46133331574419767
R^2: 0.5388855668945063

Test set:
MSE: 0.5562072457897207
R^2: 0.44240060959067595




---



---





---



---



# Catboost is best for the this problem among other compared models

In [63]:
test.drop('x_e_out [-]', axis=1)

,geometry_Unknown,geometry_annulus,geometry_plate,geometry_tube,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],D_e_ratio_D_h,length_ratio_D_h,pressure_mass_flux,length_mass_flux
4,0.0,0.0,0.0,1.0,0.777124,-1.450216,0.522908,-0.161496,-0.590050,-0.502515,-3.237903,3.653653,-1.126997,0.855700
7,0.0,0.0,0.0,1.0,1.815058,-1.411248,0.290243,-0.218275,1.277589,-0.804936,-1.329711,-5.853106,-2.561497,-1.802995
10,0.0,0.0,0.0,1.0,0.129570,0.101400,-1.423018,-0.636379,-1.067527,-0.300901,2.236118,1.677503,0.013138,-0.108247
12,1.0,0.0,0.0,0.0,-0.924003,2.698651,0.006938,-0.073746,1.715929,0.505554,-0.094076,-23.268109,-2.493562,4.630694
23,0.0,1.0,0.0,0.0,1.201173,-1.042879,-0.640417,0.050136,2.035290,-0.855339,-12.773475,40.595009,-1.252678,-2.122562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31633,0.0,0.0,0.0,1.0,0.096673,0.101400,0.607514,-0.140849,-0.269522,-0.905743,-4.313231,1.913556,0.009803,-0.027329
31634,0.0,0.0,1.0,0.0,-2.373659,-0.650156,1.347812,5.459669,-1.289828,1.211202,0.246867,-0.236247,1.543248,0.838589
31637,0.0,0.0,0.0,1.0,0.777124,-1.448998,-0.046875,-0.161496,-0.590050,-0.754532,0.290253,3.653653,-1.126051,0.854982
31640,1.0,0.0,0.0,0.0,0.777124,0.009588,-0.830779,-0.491849,-0.006403,0.051923,1.689094,0.013018,0.007451,-0.000061


In [70]:
# Make predictions on the test data
y_test_pred = catboost.predict(test)

# Create a submission dataframe
submission_df = pd.DataFrame({'x_e_out [-]': y_test_pred}, index=test.index)

# Save the submission dataframe to a CSV file
submission_df.to_csv('submission.csv', index=False)

In [71]:
submission_df

,x_e_out [-]
4,0.110523
7,-0.938276
10,-0.547289
12,0.310977
23,0.536583
...,...
31633,0.534040
31634,-0.435571
31637,0.374210
31640,-0.607336
